<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/LMQG_SquaD_1_15_End_to_End_QG_4000_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test Settings:

1. **Model**: MT5-base;
2. **Dataset**: Squad 1.15 for QG;
3. **Examples**: $4000$;
4. **Other**: End-to-End Question Generation example.

An example of QG finetuned T5 model (`t5-base`) over a variant of SquaD V1 dataset for Question Generation.

### 1. Preliminaries

#### Mounting and Installing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install "transformers[sentencepiece]"
!pip install "transformers[torch]"
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
NUM_EPOCHS = 20

#### Imports

In [ ]:
from transformers import AutoModel, AutoTokenizer, MT5ForConditionalGeneration, TrainingArguments, Trainer, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os

In [ ]:
model_checkpoint='google/mt5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
%run utils.ipynb

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f074e53a98ff4c205f399b7c64bc9f553624c29806ee0a3cac2f6385190c2b63
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00


Updated!


NOTE: `T5` uses `Negative Log Likelihood` by default.

### 2. Dataset Loading and Preprocessing

We will use [`squad_it`](https://huggingface.co/datasets/squad_it), a variant of the `SquaD` dataset adapted to `Question Generation` tasks.

In [ ]:
def get_maximum_labels_length(dataset):
  tokenized_dataset_lengths = [len(tokenizer.tokenize(sample)) for sample in dataset['questions']]
  return max(tokenized_dataset_lengths)

In [ ]:
MAX_INPUTS_LENGTH = 512
MAX_LABELS_LENGTH = 64
def tokenizer_function(samples, max_inputs_length=MAX_INPUTS_LENGTH, max_labels_length=MAX_LABELS_LENGTH, input_ids_padding=True, train_dataset=None):
  max_labels_length = max_labels_length if not train_dataset else get_maximum_labels_length(train_dataset)
  #print(max_labels_length)
  input_tokenized = tokenizer(samples['context'], padding=input_ids_padding, max_length=max_inputs_length, truncation=True, return_tensors='pt')
  labels_tokenized = tokenizer(samples['questions'], padding="max_length", max_length=max_labels_length, truncation=True, return_tensors='pt')
  labels, masks = labels_tokenized['input_ids'], labels_tokenized['attention_mask']
  argmin_masks = torch.argmin(masks, dim=-1)
  for index in range(len(argmin_masks)):
    if masks[index][argmin_masks[index]] == 0:
      labels[index][argmin_masks[index]:] = -100
  input_tokenized['labels'] = labels
  return input_tokenized

In [ ]:
def load_and_preprocess_squad_qg_dataset(
    dataset_name='derek-thomas/squad-v1.1-t5-question-generation', train_dataset_name='squad_qg_train',
    eval_dataset_name='squad_qg_eval', test_dataset_name='squad_qg_test', shuffle_seed=None,
    train_select=None, eval_select=None, use_extra_ids=False, eval_split=0.3,
):
  dataset_loading_result = get_dataset(dataset_name, train_dataset_name, eval_dataset_name, test_dataset_name)
  local = dataset_loading_result['local']
  if local:
    train_dataset = dataset_loading_result['train']
    validation_dataset = dataset_loading_result['eval']
    test_dataset = dataset_loading_result['test']
  if not local:
    datasets = load_dataset(dataset_name)
    dev_dataset, test_dataset = datasets['train'], datasets['validation']
    print(f"Dev dataset has {len(dev_dataset)} items. Test dataset has {len(test_dataset)} items.")

    eval_length = int(0.2 * len(dev_dataset))
    train_length = len(dev_dataset) - eval_length

    dev_dataset = dev_dataset.shuffle(seed=shuffle_seed)
    train_dataset = dev_dataset.select(range(train_length))
    validation_dataset = dev_dataset.select(range(train_length, train_length + eval_length))

    train_dataset.save_to_disk(train_dataset_name)
    validation_dataset.save_to_disk(eval_dataset_name)
    test_dataset.save_to_disk(test_dataset_name)

  if train_select:
    train_dataset = train_dataset.shuffle(seed=0).select(range(train_select))
  if eval_select:
    validation_dataset = validation_dataset.shuffle(seed=0).select(range(eval_select))

  print(train_dataset, validation_dataset, test_dataset)
  print(train_dataset[0])
  tokenizer_function_lambda = lambda samples: tokenizer_function(samples, max_labels_length=128)
  tokenized_train_dataset = train_dataset.map(tokenizer_function_lambda, batched=True).remove_columns(['context', 'questions'])
  tokenized_validation_dataset = validation_dataset.map(tokenizer_function_lambda, batched=True).remove_columns(['context', 'questions'])
  tokenized_test_dataset = test_dataset.map(lambda samples: tokenizer_function(samples, max_labels_length=128, input_ids_padding="max_length"), batched=True).remove_columns(['context', 'questions'])

  tokenized_train_dataset.set_format("torch")
  tokenized_validation_dataset.set_format("torch")
  tokenized_test_dataset.set_format("torch")

  return (train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset)

In [ ]:
(train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset) = \
  load_and_preprocess_squad_qg_dataset(shuffle_seed=42, train_select=4000, eval_select=2000, use_extra_ids=True)

Dataset({
    features: ['context', 'questions'],
    num_rows: 4000
}) Dataset({
    features: ['context', 'questions'],
    num_rows: 2000
}) Dataset({
    features: ['context', 'questions'],
    num_rows: 2067
})
{'context': "gq: In Fall 2008, Northwestern opened a campus in Education City, Doha, Qatar, joining five other American universities: Carnegie Mellon University, Cornell University, Georgetown University, Texas A&M University, and Virginia Commonwealth University. Through the Medill School of Journalism and School of Communication, NU-Q offers bachelor's degrees in journalism and communication respectively. The Qatar Foundation for Education, Science and Community Development provided funding for construction and administrative costs as well as support to hire 50 to 60 faculty and staff, some of whom rotate between the Evanston and Qatar campuses. In February 2016, Northwestern reached an agreement with the Qatar Foundation to extend the operations of the NU-Q branch for an

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

In [ ]:
get_maximum_labels_length(train_dataset)

308

### 3. Model Loading and configuration

#### Loading

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

#### Configuration

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
  num_training_steps = get_training_configuration(train_batch_size=2, eval_batch_size=2, tokenizer=tokenizer, learning_rate=1e-4, num_epochs=NUM_EPOCHS)

40000


### 4. Fine-tuning

#### Execution

In [ ]:
training_results_dict = main_training_loop(
    model, device, optimizer, train_dataloader, eval_dataloader,
    lr_scheduler, num_training_steps, num_epochs=NUM_EPOCHS, metrics=None,
    eval_strategy='epoch', eval_every=2000, model_save_path='squad_qg_reduced_mt5base_20epochs',
    early_stopping=True, early_stopping_patience=10, tokenizer=tokenizer,
  )

epoch_train_losses = training_results_dict['epoch_train_losses']
epoch_eval_losses = training_results_dict['epoch_eval_losses']
epoch_eval_metrics = training_results_dict['epoch_eval_metrics']
print(epoch_train_losses, epoch_eval_losses, epoch_eval_metrics, sep='\n')

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
<ipython-input-6-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 0: Train Loss = 2.9014556407928467, Eval Loss = 1.9420266151428223
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 1: Train Loss = 2.050025701522827, Eval Loss = 1.76814603805542
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-6-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


...

Epoch 10: Train Loss = 1.114994764328003, Eval Loss = 1.324202299118042
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-6-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 11: Train Loss = 1.4180840253829956, Eval Loss = 1.375915765762329
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 12: Train Loss = 0.9337133765220642, Eval Loss = 1.4362748861312866
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 13: Train Loss = 1.3814579248428345, Eval Loss = 1.3670403957366943
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-6-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 14: Train Loss = 1.2525452375411987, Eval Loss = 1.430341362953186
Save this model (y/n)?> n
Continue training (y/n)?> n
[2.9014556407928467, 2.050025701522827, 2.8065760135650635, 2.2973268032073975, 1.8933850526809692, 2.1256518363952637, 1.6613727807998657, 1.5970056056976318, 1.907239317893982, 1.44252347946167, 1.114994764328003, 1.4180840253829956, 0.9337133765220642, 1.3814579248428345, 1.2525452375411987]
[1.9420266151428223, 1.76814603805542, 1.6317739486694336, 1.5213961601257324, 1.3580423593521118, 1.5080310106277466, 1.3782634735107422, 1.399932622909546, 1.353195071220398, 1.3916352987289429, 1.324202299118042, 1.375915765762329, 1.4362748861312866, 1.3670403957366943, 1.430341362953186]
[]


In [ ]:
model.save_pretrained(f"squad_qg_reduced_mt5base_20epochsmax_(epoch {training_results_dict['epoch']})")

In [ ]:
del model
torch.cuda.synchronize()

### 5. Analysis of the Results

### Calculating BLEU and ROUGE score

In [ ]:
loss_tracker=[]
metrics_tracker=[]
metrics = {
    'bleu': get_bleu_config(tokenizer),
    'nist': get_nist_config(tokenizer),
    'rouge': get_rouge_config(tokenizer),
}

In [ ]:
P = True

In [ ]:
if P:
  model = MT5ForConditionalGeneration.from_pretrained('squad_qg_reduced_mt5base_20epochs_epoch10', local_files_only=True)

In [ ]:
if P:
  optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
    num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-3, num_epochs=2)

2000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
if P:
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)
  print()

In [ ]:
tokenized_test_dataset_reduced = tokenized_test_dataset.shuffle(seed=42).select(range(400))
test_dataset_reduced = test_dataset.shuffle(seed=42).select(range(400))

In [ ]:
tokenized_test_dataset_reduced = tokenized_test_dataset
test_dataset_reduced = test_dataset

In [ ]:
len(test_dataset)

2067

In [ ]:
from tqdm.auto import tqdm
test_dataloader = DataLoader(tokenized_test_dataset_reduced, shuffle=True, batch_size=8, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
test_loss_tracker, test_metrics_tracker, num_test_steps = [], [], len(test_dataloader)
test_progress_bar = tqdm(range(num_test_steps))
test_loss = evaluation_loop(
    model, device, optimizer, test_dataloader, lr_scheduler, test_loss_tracker, test_metrics_tracker, metrics, test_progress_bar,
    tokenizer=tokenizer, num_beams=4, num_candidates=4, tokenize_predictions_output=False,
)

  0%|          | 0/259 [00:00<?, ?it/s]

<ipython-input-8-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Metrics = {'bleu': {'bleu': 0.2084466073164407, 'precisions': [0.5169388252498172, 0.30086777701942086, 0.1911331828442438, 0.11217025661787032], 'brevity_penalty': 0.8674364599421139, 'length_ratio': 0.8754934386002348, 'translation_length': 114884, 'reference_length': 131222}, 'nist': {'nist_mt': 4.242865063751756}, 'rouge': {'rouge1': 0.4332176969426049, 'rouge2': 0.19816097850828915, 'rougeL': 0.3558377953216775, 'rougeLsum': 0.3559039361830537}}


In [ ]:
import json
with open('squad_qg_reduced_mt5_base_epoch10_test_results.json', 'w') as out_file:
  json.dump({'loss': test_loss_tracker, 'metrics': test_metrics_tracker}, out_file)

In [ ]:
test_dataset[18:20]['questions']

['Question: Who has the record of being the oldest quarter back in any Super Bowl game? Question: Who is the oldest quarterback to play in a Super Bowl? Question: Who was the first quarterback to take two teams to more than one Super Bowl? Question: How many teams has Manning played for that reached the Super Bowl, while he was on their team? Question: Peyton Manning took how many different teams to the Super Bowl? Question: How old was Peyton Manning when he played in Super Bowl 50? Question: How old was Manning when he played Super Bowl 50? Question: Who previously held the record for being the oldest quarterback to play in a Super Bowl? Question: Who is the General Manager for the Broncos? Question: Prior to Manning, who was the oldest quarterback to play in a Super Bowl? Question: What is the name of the quarterback who was 38 in Super Bowl XXXIII? Question: Who did John Elway play for in Super Bowl XXXIII? Question: What team was the winner of Super Bowl XXXIII? Question: Which Su

In [ ]:
model.eval()
with torch.no_grad():
  predictions = model.generate(tokenized_test_dataset['input_ids'][18:20].to(device), max_length=64, num_beams=4)

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

["Question: Who became the first quarterback ever to play in a Super Bowl? Question: What is the oldest quarterback ever to play in a Super Bowl? Question: What is the oldest quarterback ever to play in a Super Bowl? Question: Who is currently Denver'",
 'Question: When was the first Super Bowl to feature a quarterback on both teams? Question: When was the first Super Bowl to feature a quarterback on both teams? Question: When was the first Super Bowl to feature a quarterback on both teams? Question: How']

In [ ]:
test_dataset[108:120]['question']

['When was slavery completely outlawed in the state of New York?',
 'The idea that people are unchanging and stay the same even through changes is considered what?',
 'What can be worn in cold winter weather?',
 'Where did Chopin debut after completing his studies?',
 "Who was Alfonso III's third son and what area did he receive?",
 'What river was adjacent to HMNB Davenport?',
 'In what city was the 1966 NABBA Mr. Universe competition held?',
 "How much did Chopin's funeral cost?",
 'Afonso heard Jesus promising what?',
 'What was the last year that a republican candidate won all four boroughs of NYC?',
 'Who used the the Ordos region as a place to stage raids?',
 'How large was the number of injured in Beichuan County?']

In [ ]:
model.eval()
with torch.no_grad():
  predictions = model.generate(tokenized_test_dataset['input_ids'][108:120].to(device), max_length=64, num_beams=4)

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

['What is the largest private foundation in the world?',
 'What does a dialect continuum a dialect continuum?',
 'When did the British Isles become a part of the British Isles?',
 'What was the name of the first film Madonna directed with Adam Lambert?',
 'What was the name of the famous cleric during the early 20th century?',
 'When did the British Isles become a part of the empire?',
 'When did the British Isles become a part of the empire?',
 'What was the name of the famous cleric during the early 20th century?',
 'What was the name of the famous cleric during the early 20th century?',
 'When was the Treaty of Sevres signed?',
 'When did the British Isles become a part of the empire?',
 'What is the largest private foundation in the world?']

In [ ]:
test_dataset[990:1000]['question']

['When was Northern Rock taken into public hands?',
 'How many people work in the New York publishing industry?',
 'Until when did the Portuguese government resist decolonization of their overseas territories?',
 'Who gave Chopin a loan in September for an apartment?',
 'What was the U.S. unemployment rate in October 2009?',
 "Which team did Notre Dame's football team find inspiration from?",
 'What countries used comprehensive schools extensively?',
 'Who was the mayor of Nagano?',
 'Who did Ü-Tsang king have an alliance with?',
 'At what age did Kanye West relocate to China?']

In [ ]:
model.eval()
with torch.no_grad():
  predictions = model.generate(tokenized_test_dataset['input_ids'][990:1000].to(device), max_length=64, num_beams=10)

OutOfMemoryError: ignored

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

['What is the largest private foundation in the world?',
 'What does a dialect continuum a dialect continuum?',
 'When did the British Isles become a part of the British Isles?',
 'What was the name of the first film Madonna directed with Adam Lambert?',
 'What was the name of the famous cleric during the early 20th century?',
 'When did the British Isles become a part of the empire?',
 'When did the British Isles become a part of the empire?',
 'What was the name of the famous cleric during the early 20th century?',
 'What was the name of the famous cleric during the early 20th century?',
 'When was the Treaty of Sevres signed?',
 'When did the British Isles become a part of the empire?',
 'What is the largest private foundation in the world?']